# 05a - Vertex AI > Training > Custom Jobs - With Python file

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05a`): An extension of `05` that saves the notebook code for the model as a Python script
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script.  The script is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service. This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob.from_local_script()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob.from_local_script).

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05a...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
    - Pre-built Containers for Vertex AI
        - [Training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
        - [Prediction & Explaination](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)





---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05a_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05a_console.png">

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
DATANAME = 'fraud'
NOTEBOOK = '05a'
SERIES = '05'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [3]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [4]:
aiplatform.init(project = PROJECT_ID, location = REGION)
bigquery = bigquery.Client()

parameters:

In [5]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [6]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

Experiment Tracking:

In [8]:
TASK = 'classification'
MODEL_TYPE = 'dnn'
EXPERIMENT_NAME = f'{SERIES}-{NOTEBOOK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [9]:
tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

In [10]:
tb.resource_name

'projects/1026793852137/locations/us-central1/tensorboards/1212224763762573312'

---
## Setup Vertex AI Experiments

The code in this section initializes the experiment and starts a run that represents this notebook.  Throughout the notebook sections for model training and evaluation information will be logged to the experiment using:
- [.log_params](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_params)
- [.log_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_metrics)
- [.log_time_series_metrics](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform#google_cloud_aiplatform_log_time_series_metrics)

In [12]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

In [13]:
aiplatform.start_run(run = RUN_NAME)

Associating projects/1026793852137/locations/us-central1/metadataStores/default/contexts/05-05a-classification-dnn-run-20220818224910 to Experiment: 05-05a-classification-dnn


In [14]:
aiplatform.log_params({'DATANAME': DATANAME, 'NOTEBOOK': NOTEBOOK, 'SERIES': SERIES, 'PROJECT_ID': PROJECT_ID, 'VAR_TARGET': VAR_TARGET})

In [15]:
aiplatform.end_run()

In [16]:
aiplatform.get_experiment_df()

,experiment_name,run_name,run_type,state,param.NOTEBOOK,param.SERIES,param.DATANAME,param.VAR_TARGET,param.PROJECT_ID,param.MODEL_DISPLAY_NAME,...,metric.val_auprc,metric.test_accuracy,metric.val_loss,metric.test_auprc,time_series_metric.val_loss,time_series_metric.val_auprc,time_series_metric.train_auprc,time_series_metric.val_accuracy,time_series_metric.train_accuracy,time_series_metric.train_loss
0,05-05a-classification-dnn,run-20220818212823,system.ExperimentRun,COMPLETE,05a,05,fraud,Class,statmike-mlops-349915,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,05-05a-classification-dnn,run-20220818191710,system.ExperimentRun,COMPLETE,05a,05,fraud,Class,statmike-mlops-349915,05a_fraud,...,0.999627,0.999299,0.004269,0.999628,0.004269,0.999627,0.999709,0.999368,0.999359,0.003268


---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [16]:
%%writefile {DIR}/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
from google.cloud import aiplatform
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
parser.add_argument('--experiment_name', dest = 'experiment_name', type=str)
parser.add_argument('--run_name', dest = 'run_name', type=str)
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)
aiplatform.init(project = PROJECT_ID, location = REGION)

# Vertex AI Experiment
expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]
expRun.log_params({'data_source': f'bq://{DATANAME}.{DATANAME}_prepped', 'nclasses': nclasses, 'var_split': 'splits'})

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict,target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
expRun.log_params({'BATCH_SIZE': args.batch_size, 'SHUFFLE': 10*args.batch_size, 'PREFETCH': 1})

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values(), name = 'feature_layer')(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization(name = 'batch_normalization_layer')(feature_layer_outputs)
layers = tf.keras.layers.Dense(64, activation = 'relu', name = 'hidden_layer')(layers)
layers = tf.keras.layers.Dense(32, activation = 'relu', name = 'embedding_layer')(layers)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax, name = 'prediction_layer')(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers,
    name = f'{DATANAME}'
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR', name = 'auprc')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback], validation_data = validate)
expRun.log_params({'epochs': history.params['epochs']})
for e in range(0, history.params['epochs']):
    expRun.log_time_series_metrics(
        {
            'train_loss': history.history['loss'][e],
            'train_accuracy': history.history['accuracy'][e],
            'train_auprc': history.history['auprc'][e],
            'val_loss': history.history['val_loss'][e],
            'val_accuracy': history.history['val_accuracy'][e],
            'val_auprc': history.history['val_auprc'][e]
        }
    )

# evaluations:
loss, accuracy, auprc = model.evaluate(test)
expRun.log_metrics({'test_loss': loss, 'test_accuracy': accuracy, 'test_auprc': auprc})
loss, accuracy, auprc = model.evaluate(validate)
expRun.log_metrics({'val_loss': loss, 'val_accuracy': accuracy, 'val_auprc': auprc})
loss, accuracy, auprc = model.evaluate(train)
expRun.log_metrics({'train_loss': loss, 'train_accuracy': accuracy, 'train_auprc': auprc})

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))
expRun.log_params({'MODEL_URI': os.getenv("AIP_MODEL_DIR")})
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

Overwriting temp/05a/train.py


### Setup Training Job

In [17]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME
]

In [18]:
customJob = aiplatform.CustomJob.from_local_script(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    script_path = f"{DIR}/train.py",
    container_uri = TRAIN_IMAGE,
    args = CMDARGS,
    requirements = ['tensorflow_io', f'google-cloud-aiplatform>={aiplatform.__version__}'],
    replica_count = 1,
    machine_type = TRAIN_COMPUTE,
    accelerator_count = 0,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

Training script copied to:
gs://statmike-mlops-349915/fraud/models/05a/20220818224910/aiplatform-2022-08-18-22:50:13.891-aiplatform_custom_trainer_script-0.1.tar.gz.


### Run Training Job

In [19]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

Creating CustomJob
CustomJob created. Resource name: projects/1026793852137/locations/us-central1/customJobs/4240479073140211712
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/1026793852137/locations/us-central1/customJobs/4240479073140211712')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4240479073140211712?project=1026793852137
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+1212224763762573312+experiments+4240479073140211712
CustomJob projects/1026793852137/locations/us-central1/customJobs/4240479073140211712 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/4240479073140211712 current state:
JobState.JOB_STATE_PENDING
CustomJob projects/1026793852137/locations/us-central1/customJobs/4240479073140211712 current state:
JobState.JOB_STATE_PENDING


In [23]:
customJob.display_name

'05a_fraud_20220818212823'

In [24]:
customJob.resource_name

'projects/1026793852137/locations/us-central1/customJobs/3847821480628846592'

log parameters, create hyperlines to job and tensorboard here:

In [21]:
expRun = aiplatform.ExperimentRun(run_name = RUN_NAME, experiment = EXPERIMENT_NAME)

In [22]:
exp = aiplatform.Experiment(experiment_name = EXPERIMENT_NAME)

In [23]:
expRun.log_params({
    'JobType': 'aiplatform.CustomJob.from_local_script()',
    'CustomJob.display_name': customJob.display_name,
    'CustomJob.resource_name': customJob.resource_name
})

In [24]:
print(f"Review the Job here:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/training/{customJob.resource_name.split('/')[-1]}/cpu?cloudshell=false&project={PROJECT_ID}")

Review the Job here:
https://console.cloud.google.com/vertex-ai/locations/us-central1/training/4240479073140211712/cpu?cloudshell=false&project=statmike-mlops-349915


In [25]:
print(f"Review the TensorBoard From the Job here:\nhttps://{REGION}.tensorboard.googleusercontent.com/experiment/{tb.resource_name.replace('/', '+')}+experiments+{customJob.resource_name.split('/')[-1]}")

Review the TensorBoard From the Job here:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+1212224763762573312+experiments+4240479073140211712


---
## Serving

### Upload The Model

In [26]:
modelmatch = aiplatform.Model.list(filter = f'display_name="{NOTEBOOK}_{DATANAME}"')
if modelmatch:
    print("Model Already in Registry:")
    if f'time-{TIMESTAMP}' in modelmatch[0].version_aliases:
        print("This version already loaded, no action taken.")
        model = aiplatform.Model(model_name = modelmatch[0].resource_name)
    else:
        print('Loading model as new default version.')
        model = aiplatform.Model.upload(
            display_name = f'{NOTEBOOK}_{DATANAME}',
            model_id = f'model_{NOTEBOOK}_{DATANAME}',
            parent_model =  modelmatch[0].resource_name,
            serving_container_image_uri = DEPLOY_IMAGE,
            artifact_uri = f"{URI}/{TIMESTAMP}/model",
            is_default_version = True,
            version_aliases = [f'run-{TIMESTAMP}'],
            version_description = f'run-{TIMESTAMP}',
            labels = {'notebook':f'{NOTEBOOK}'}        
        )
else:
    print('This is a new model, creating in model registry')
    model = aiplatform.Model.upload(
        display_name = f'{NOTEBOOK}_{DATANAME}',
        model_id = f'model_{NOTEBOOK}_{DATANAME}',
        serving_container_image_uri = DEPLOY_IMAGE,
        artifact_uri = f"{URI}/{TIMESTAMP}/model",
        is_default_version = True,
        version_aliases = [f'run-{TIMESTAMP}'],
        version_description = f'run-{TIMESTAMP}',
        labels = {'notebook':f'{NOTEBOOK}'}
    )    

Model Already in Registry:
Loading model as new default version.
Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/model_05a_fraud/operations/7689500883146506240
Model created. Resource name: projects/1026793852137/locations/us-central1/models/2624978458198933504@8
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/2624978458198933504@8')


**Note** on Version Aliases:
>Expectation is a name starting with `a-z` that can include `[a-zA-Z0-9-]`

**Retrieve a Model Resource**

[Resource](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)
```Python
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}') # retrieves default version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}@time-{TIMESTAMP}') # retrieves specific version
model = aiplatform.Model(model_name = f'model_{NOTEBOOK}_{DATANAME}', version = f'time-{TIMESTAMP}') # retrieves specific version
```

### Vertex AI Experiment Update and Review

In [27]:
expRun.log_params({'MODEL_DISPLAY_NAME': model.display_name, 'MODEL_VERSIONED_RESOURCE_NAME': model.versioned_resource_name})

In [28]:
expRun.update_state(state = aiplatform.gapic.Execution.State.COMPLETE)

In [29]:
exp.get_data_frame()

,experiment_name,run_name,run_type,state,param.VAR_TARGET,param.epochs,param.JobType,param.CustomJob.display_name,param.DATANAME,param.SERIES,...,metric.test_auprc,metric.test_loss,metric.val_loss,metric.train_auprc,time_series_metric.val_loss,time_series_metric.train_loss,time_series_metric.train_auprc,time_series_metric.train_accuracy,time_series_metric.val_auprc,time_series_metric.val_accuracy
0,05-05a-classification-dnn,run-20220818224910,system.ExperimentRun,COMPLETE,Class,10.0,aiplatform.CustomJob.from_local_script(),05a_fraud_20220818224910,fraud,05,...,0.999627,0.003938,0.004302,0.999662,0.004302,0.003036,0.999744,0.999416,0.999580,0.999438
1,05-05a-classification-dnn,run-20220818212823,system.ExperimentRun,COMPLETE,Class,10.0,aiplatform.CustomJob.from_local_script(),05a_fraud_20220818212823,fraud,05,...,0.999580,0.004556,0.004658,0.999579,0.004658,0.003162,0.999709,0.999429,0.999487,0.999368
2,05-05a-classification-dnn,run-20220818191710,system.ExperimentRun,COMPLETE,Class,10.0,aiplatform.CustomJob.from_local_script(),05a_fraud_20220818191710,fraud,05,...,0.999628,0.004074,0.004269,0.999639,0.004269,0.003268,0.999709,0.999359,0.999627,0.999368


### Create An Endpoint

In [331]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
if endpoints:
    endpoint = endpoints[0]
    print(f"Endpoint Exists: {endpoints[0].resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(
        display_name = f"{SERIES}_{DATANAME}",
        labels = {'notebook':f"{SERIES}"}    
    )
    print(f"Endpoint Created: {endpoint.resource_name}")

Endpoint Exists: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952


In [332]:
endpoint.display_name

'05_fraud'

### Deploy Model To Endpoint

In [333]:
dmodels = endpoint.list_models()

check = 0
if dmodels:
    for dmodel in dmodels:
        if dmodel.model == model.resource_name and dmodel.model_version_id == model.version_id and dmodel.id in endpoint.traffic_split:
            print(f'This model (and version) already deployed with {endpoint.traffic_split[dmodel.id]}% of traffic')
            check = 1
    
if check == 0:
    print(f'Deploying model with 100% of traffic...')
    endpoint.deploy(
        model = model,
        deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}',
        traffic_percentage = 100,
        machine_type = DEPLOY_COMPUTE,
        min_replica_count = 1,
        max_replica_count = 1
    )     

Deploying model with 100% of traffic...
Deploying Model projects/1026793852137/locations/us-central1/models/2624978458198933504 to Endpoint : projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952/operations/8996115696496672768


/opt/conda/lib/python3.7/site-packages/proto/marshal/rules/enums.py:40: UserWarning: Unrecognized DeploymentResourcesType enum value: 3
  value=value,


Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952


### Remove Deployed Models without Traffic

In [334]:
for dmodel in endpoint.list_models():
    if dmodel.id in endpoint.traffic_split:
        print(f"Model {dmodel.display_name} has traffic = {endpoint.traffic_split[dmodel.id]}")
    else:
        endpoint.undeploy(deployed_model_id = dmodel.id)
        print(f"Undeployed {dmodel.display_name} version {dmodel.model_version_id} as it has no traffic.")

Model 05a_fraud has traffic = 100
Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952/operations/7122618251510546432
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/7252545822577917952
Undeployed 05a_fraud version 4 as it has no traffic.


In [335]:
endpoint.traffic_split

{'1628935272643166208': 100}

In [336]:
endpoint.list_models()

[id: "1628935272643166208"
 model: "projects/1026793852137/locations/us-central1/models/model_05a_fraud"
 display_name: "05a_fraud"
 create_time {
   seconds: 1659050517
   nanos: 981042000
 }
 dedicated_resources {
   machine_spec {
     machine_type: "n1-standard-4"
   }
   min_replica_count: 1
   max_replica_count: 1
 }
 model_version_id: "5"]

---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [337]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [338]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


In [339]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [340]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [341]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999751031, 0.000248962489]], deployed_model_id='1628935272643166208', model_version_id='', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05a_fraud', explanations=None)

In [342]:
prediction.predictions[0]

[0.999751031, 0.000248962489]

In [343]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [344]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [345]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999751031,
      0.000248962489
    ]
  ],
  "deployedModelId": "1628935272643166208",
  "model": "projects/1026793852137/locations/us-central1/models/model_05a_fraud",
  "modelDisplayName": "05a_fraud"
}


### Get Predictions: gcloud (CLI)

In [346]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999751031, 0.000248962489]]


---
## Remove Resources
see notebook "99 - Cleanup"